In [1]:
import numpy as np
import pandas as pd

from funcs.model import FlareModulator

import astropy.units as u
from astropy.constants import R_sun

torad = lambda x: x / 180. * np.pi

import matplotlib.pyplot as plt
%matplotlib inline

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
NumExpr defaulting to 8 threads.


In [16]:
# Get best-fit results
# _deprecated
df = pd.read_csv("../results/mcmc/15_12_2020_GP_mcmcoutput.csv")

# Pick up stellar parameters
props = pd.read_csv(f"../../data/summary/lcs.csv")

# Get MCMC inputs
mcmcinit = pd.read_csv(f"../../data/summary/inits_decoupled_GP.csv")

In [17]:
df

,ED_s_16,ED_s_50,ED_s_84,Eflare_erg_16,Eflare_erg_50,Eflare_erg_84,ID,a_16,a_50,a_84,...,phase_deg_50,phase_deg_84,rad_rsun_16,rad_rsun_50,rad_rsun_84,suffix,t0_d_16,t0_d_50,t0_d_84,tstamp
0,2489.574477,2613.735051,2761.358049,3.236447e+33,3.397856e+33,3.589765e+33,452922110,0.425608,0.468987,0.510600,...,298.215052,304.049226,8.392326,8.812874,9.198803,NaN,1493.930922,1493.931292,1493.931683,10_12_2020_07_05
1,21765.855753,26408.882022,30825.614977,2.451641e+34,2.974618e+34,3.472105e+34,277539431,2.140930,2.609415,3.055521,...,232.093998,234.611761,16.743598,18.545130,20.130920,NaN,1641.838917,1641.840286,1641.841430,07_12_2020_15_26
2,4211.035749,4481.584862,4849.726702,3.735479e+34,3.975475e+34,4.302042e+34,237880881,2.138036,2.328796,2.571447,...,143.960000,147.306157,25.239164,26.424194,27.880130,a,1331.665643,1331.665941,1331.666255,10_12_2020_07_11
3,5113.166019,5268.604916,5592.077901,4.535731e+34,4.673616e+34,4.960559e+34,237880881,0.571344,0.597261,0.637065,...,143.960000,147.306157,12.733859,13.024430,13.459331,b,1331.827605,1331.828257,1331.828904,10_12_2020_07_11
4,2399.682955,2433.253326,2466.289728,2.233733e+33,2.264982e+33,2.295734e+33,44984200,0.493929,0.504206,0.514489,...,257.654681,261.565710,7.373422,7.450160,7.526186,a,1588.010945,1588.011745,1588.012171,10_12_2020_07_10
5,89.235046,105.973394,123.505312,8.306400e+31,9.864481e+31,1.149643e+32,44984200,0.064858,0.077785,0.091753,...,257.654681,261.565710,2.665475,2.919259,3.170797,b,1588.117627,1588.118865,1588.120303,10_12_2020_07_10


In [8]:
row = df.iloc[-2]
row

ED_s_16                     2455.51
ED_s_50                     2484.32
ED_s_84                     2513.69
Eflare_erg_16            2.2857e+33
Eflare_erg_50           2.31251e+33
Eflare_erg_84           2.33986e+33
ID                         44984200
a_16                       0.319331
a_50                       0.323594
a_84                        0.32798
color                             r
frac_area_16             0.00266913
frac_area_50             0.00270487
frac_area_84             0.00274163
fwhm1_d_16                0.0343652
fwhm1_d_50                0.0349237
fwhm1_d_84                0.0354754
fwhm2_d_16                0.0591695
fwhm2_d_50                0.0600101
fwhm2_d_84                0.0608576
i_deg_16                    32.2363
i_deg_50                    33.0831
i_deg_84                    33.9327
latitude_deg_16              82.416
latitude_deg_50             83.2479
latitude_deg_84             84.0825
linestyle                     solid
phase_deg_16                

# 1-flare solution

In [9]:
# get the right suffix
if np.isnan(row.suffix):
    row.suffix = ""
    
# row.suffix="c"

# Pick your star
target = props[props.ID == int(row.ID)].iloc[0]

# get qlum and R as they were passed to MCMC
inits = mcmcinit[(mcmcinit.ID.astype(str) == str(row.ID) + str(row.suffix)) &
                 (mcmcinit.tstamp == row.tstamp)].iloc[0]

qlum = inits.qlum_erg_s * u.erg / u.s
R = inits.R_Rsun * R_sun

# Get light curve
lc = pd.read_csv(f"../../data/lcs/{row.tstamp}_{row.ID}.csv")

# Get flux median
median = lc.median_[0]

# Convert MCMC outputs to model inputs

# Get peak time in phase units
phi_a = (row.t0_d_50 -lc.t[0]) / target.Prot_d * 2 * np.pi 

# Convert flare peak phase to phase at t0
phi0 = phi_a % (2. * np.pi) - (row.phase_deg_50  / 180 * np.pi)

# Get FHWM values in phase units
fwhm1 = row.fwhm1_d_50 / target.Prot_d * 2 * np.pi
fwhm2 = row.fwhm2_d_50 / target.Prot_d * 2 * np.pi

# Init FlareModulator object
F =  FlareModulator(lc.phi.values, lc.flux.values, lc.flux_err.values, qlum=qlum, R=R,
                 median=median, nflares=1, iscoupled=False)

# Assemble best-fit parameters and constants
params = [torad(row.latitude_deg_50),
          phi0,
          torad(row.i_deg_50),
          row.a_50, 
          phi_a, 
          fwhm1, 
          fwhm2]

# Get log-likelihood for best-fiit solution
logL = F.log_likelihood(params, sta=270) # the closer to 0 the better, always negative

# Calculate Chi^2 value
chi2 = F.chi_square(params, sta=270)

# Calculate Bayesian information criterion
# second term is always positive, 
# light curve length stays constant across different fits
# number of parameters varies with 1- vs. 2-flare solution
BIC = len(params) * np.log(len(lc.flux[270:])) - 2 * logL 

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
print(f"Chi^2 for TIC {row.ID}, fit with 1 flare fit: {chi2} ({row.tstamp})")

Chi^2 for TIC 44984200, fit with 1 flare fit: 55.61939780349556 (07_12_2020_18_46)


In [11]:
print(f"BIC for TIC {row.ID}, fit with 1 flare fit: {BIC} ({row.tstamp})")

BIC for TIC 44984200, fit with 1 flare fit: 3139.305406056544 (07_12_2020_18_46)


Chi^2 for TIC 44984200, fit with 1 flare fit: 23.34274209483541 (10_12_2020_07_12) high uncert.

BIC for TIC 44984200, fit with 1 flare fit: 2509.7787794513647 (10_12_2020_07_12)


Chi^2 for TIC 44984200, fit with 1 flare fit: 55.61939780349556 (07_12_2020_18_46) normal uncert.

BIC for TIC 44984200, fit with 1 flare fit: 3139.305406056544 (07_12_2020_18_46)



Chi^2 for TIC 44984200, fit with 2 flare fit: 32.076975043748504 (07_12_2020_19_42)

BIC for TIC 44984200, fit with 2 flare fit: 3962.486746433922 (07_12_2020_19_42) normal. uncert


Chi^2 for TIC 44984200, fit with 2 flare fit: 34.245377521485224 (10_12_2020_07_10)

BIC for TIC 44984200, fit with 2 flare fit: 4003.023462928036 (10_12_2020_07_10) high uncert.


In [ ]:
plt.plot(lc.flux[270:])

# 2-flare solution

In [12]:
df.suffix

0    NaN
1      a
2      b
3      a
4      b
5    NaN
6      c
Name: suffix, dtype: object

In [18]:
# get the right suffix
# if np.isnan(row.suffix):
#     row.suffix = ""
fp = []

for j, row in df.iloc[4:6].iterrows():

    # Pick your star
    target = props[props.ID == int(row.ID)].iloc[0]

    # get qlum and R as they were passed to MCMC
    inits = mcmcinit[(mcmcinit.ID.astype(str) == str(row.ID) + str(row.suffix)) &
                     (mcmcinit.tstamp == row.tstamp)].iloc[0]

    qlum = inits.qlum_erg_s * u.erg / u.s
    R = inits.R_Rsun * R_sun

    # Get light curve
    lc = pd.read_csv(f"../../data/lcs/{row.tstamp}_{row.ID}.csv")

    # Get flux median
    median = lc.median_[0]


    # Init FlareModulator object
    F =  FlareModulator(lc.phi.values, lc.flux.values, lc.flux_err.values, qlum=qlum, R=R,
                     median=median, nflares=2, iscoupled=False)

    # Convert MCMC outputs to model inputs

    # Get peak time in phase units
    phi_a = (row.t0_d_50 -lc.t[0]) / target.Prot_d * 2 * np.pi 

    # Convert flare peak phase to phase at t0
    phi0 = phi_a % (2. * np.pi) - (row.phase_deg_50  / 180 * np.pi)

    # Get FHWM values in phase units
    fwhm1 = row.fwhm1_d_50 / target.Prot_d * 2 * np.pi
    fwhm2 = row.fwhm2_d_50 / target.Prot_d * 2 * np.pi
    
    fp.append(row.a_50)
    fp.append(phi_a)
    fp.append(fwhm1)
    fp.append(fwhm2)


# Assemble best-fit parameters and constants
params = [torad(row.latitude_deg_50),
          phi0,
          torad(row.i_deg_50)]

params =  params + fp
params

[1.2541120880055259,
 0.583489586981953,
 0.5775918556362263,
 0.504205797641469,
 11.690563334020425,
 0.5595435680126889,
 2.4809516581434083,
 0.07778486941329299,
 17.646782710634568,
 0.9827781499732255,
 0.14992688931217926]

In [19]:
# Get log-likelihood for best-fiit solution
logL = F.log_likelihood(params, sta=270) # the closer to 0 the better, always negative

# Calculate Chi^2 value
chi2 = F.chi_square(params, sta=270)


# Calculate Bayesian information criterion
# second term is always positive, 
# light curve length stays constant across different fits
# number of parameters varies with 1- vs. 2-flare solution
BIC = len(params) * np.log(len(lc.flux[270:])) - 2 * logL 

In [20]:
print(f"BIC for TIC {row.ID}, fit with 2 flare fit: {BIC} ({row.tstamp})")
print(f"Chi^2 for TIC {row.ID}, fit with 2 flare fit: {chi2} ({row.tstamp})")

BIC for TIC 44984200, fit with 2 flare fit: 4003.023462928036 (10_12_2020_07_10)
Chi^2 for TIC 44984200, fit with 2 flare fit: 34.245377521485224 (10_12_2020_07_10)


### TIC 452

**final**: BIC for TIC 452922110, fit with 1 flare fit: 3383.8764766286595
final: Chi^2 for TIC 452922110, fit with 1 flare fit: 114.56119373881997 (10_12_2020_07_05)

deprecrated: BIC for TIC 452922110, fit with 1 flare fit: 3539.4625885910755
**deprecated**: Chi^2 for TIC 452922110, fit with 1 flare fit: 105.50394277965948 (07_12_2020_15_44)

### TIC 277

final: BIC for TIC 277539431, fit with 1 flare fit: 5975.642718484921
final: Chi^2 for TIC 277539431, fit with 1 flare fit: 85.35280228125123 (07_12_2020_15_26)


### TIC 449

deprecated: BIC for TIC 44984200 (normal_uncertainty), fit with 1 flare fit: 6239.3862509798755
deprecated: Chi^2 for TIC 44984200 (normal_uncertainty), fit with 1 flare fit: 147.57585678156408 (07_12_2020_18_46)

**deprecated**: BIC for TIC 44984200 (c, high_uncertainties_peak1), fit with 1 flare fit: 4484.622756262301
deprecated: Chi^2 for TIC 44984200, fit with 1 flare fit: 144.13400509979132 (10_12_2020_07_12)

deprecated: BIC for TIC 44984200, fit with 2 flare fit: 6741.812040032524 (07_12_2020_19_42)
**deprecated**: Chi^2 for TIC 44984200, fit with 2 flare fit: 107.3520562886087 (07_12_2020_19_42)

BIC for TIC 44984200, fit with 2 flare fit: 5781.215417165257 (10_12_2020_07_10)
Chi^2 for TIC 44984200, fit with 2 flare fit: 255.0029659518811 (10_12_2020_07_10)


### TIC 237

**final**: BIC for TIC 237880881, fit with 2 flare fit: 17316.74773510112 (10_12_2020_07_11)
**final**: Chi^2 for TIC 237880881, fit with 2 flare fit: 1523.8682823327727 (10_12_2020_07_11)

deprecated: BIC for TIC 237880881, fit with 2 flare fit: 97020.40734765412  (07_12_2020_07_47)
deprecated: Chi^2 for TIC 237880881, fit with 2 flare fit: 3337.264890932395 (07_12_2020_07_47)


In [ ]:
0.0044 / 0.1446

In [ ]:
0.0084 / 0.2746

##### 